In [1]:
# Imports
import glob
import os
import string
import unicodedata

# Sklearn imports
import pandas
import sklearn
import sklearn.linear_model
import sklearn.svm
import sklearn.ensemble

In [ ]:
len(section_break_positions['/data/workspace/lexpredict-contraxsuite-samples/agreements/software_license/1582586_2015-08-31_SOFTWARE LICENSE AND ROYALTY AGREEMENT.txt'])

In [2]:
def build_document_distribution(text, characters=string.printable, norm=True):
    """
    Build document distribution based on fixed character and optionally norm to unit.
    """
    # Build character vector
    v = {}
    for c in characters:
        v["doc_char_{0}".format(c)] = text.count(c)
        v["doc_startchar_{0}".format(c)] = 0
    v["doc_startchar_other"] = 0

    # Build line start vector
    for line in text.splitlines():
        if len(line.strip()) > 0:
            c = line.strip()[0]

            if c in characters:
                v["doc_startchar_{0}".format(c)] += 1
            else:
                v["doc_startchar_other"] += 1
        else:
            continue
            
    # Norm if requested
    if norm:
        total_char = float(sum([b for a,b in v.items() if a.startswith("doc_char")]))
        total_startchar = float(sum([b for a,b in v.items() if a.startswith("doc_startchar")]))

        for k in v.keys():
            if k.startswith("doc_char"):
                v[k] = v[k] / total_char
            elif k.startswith("doc_startchar"):
                v[k] = v[k] / total_startchar

    return  v

In [ ]:
# Model parameters
line_window_pre = 3
line_window_post = 3

# Setup feature and target data
feature_data = []
target_data = []

# Iterate through files and test
for file_name in sorted(list(section_break_positions.keys())):
    # Read and get doc distribution
    file_buffer = open(file_name, "rb").read().decode("utf-8")
    doc_distribution=build_document_distribution(file_buffer)
    
    # Split to lines and iterate
    file_lines = file_buffer.splitlines()
    for line_id in range(len(file_lines)):
        feature_data.append(build_section_break_features(file_lines, line_id, line_window_pre, line_window_post, include_doc=doc_distribution))
        target_data.append(1 if (line_id) in section_break_positions[file_name] else 0)
        if target_data[-1] == 1:
            #print((file_name, line_id, target_data[-1]))
            #print(file_lines[line_id])
            pass

# Convert to DF
feature_df = pandas.DataFrame(feature_data).fillna(-1)